<!-- Autoheader begin -->
<hr/>
<div id="navtitle_3_3_py" style="text-align:center; font-size:16px">III.3 Using Krotov's method to separate chiral molecules</div>
<hr/>
<table style="width: 100%">
  <tr>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_2_lambda.ipynb">$\leftarrow$ previous notebook </a><br>
        <a href="py_exercise_3_2_lambda.ipynb" style="font-size:13px">III.2 Optimal Control for STIRAP</a>
    </th>
    <td style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_2_3_chiral.ipynb">$\uparrow$ previous part $\uparrow$</a><br>
        <a href="py_exercise_2_3_chiral.ipynb" style="font-size:13px">II.3 Parameter Optimization of Three-Wave Mixing in a Three-Level System</a>
    </td>
    <th rowspan="2" style="width:33%; text-align:center; font-size:16px">
        <a href="py_exercise_3_4_gate.ipynb">next notebook $\rightarrow$</a><br>
        <a href="py_exercise_3_4_gate.ipynb" style="font-size:13px">III.4 Entangling Quantum Gates for Coupled Transmon Qubits</a>
    </th>
  </tr>
  <tr style="width: 100%">
    <td style="width:33%; text-align:center; font-size:16px">
    </td>
  </tr>
</table>

<div style="text-align: right;font-size: 16px"><a href="../Julia/jl_exercise_3_3_chiral.ipynb">👉 Julia version</a></div>

---
<!-- Autoheader end -->

# Using Krotov's method to separate chiral molecules

In this notebook, you will run a gradient-based
optimization for three-wave mixing in a chiral three-level system,
building up on [Exercise II.3](py_exercise_2_3_chiral.ipynb).
We keep the same goal of driving two three-level systems,
describing the quantum states of the enantiomers of a chiral molecule,
such that they end up in distinct final states, but use a gradient-based optimization method instead, specifically Krotov's method.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import krotov
import qutip
sqrt2 = np.sqrt(2)
pi = np.pi

from utils.exercise_3_chiral import *

$\newcommand{tr}[0]{\operatorname{tr}}
\newcommand{diag}[0]{\operatorname{diag}}
\newcommand{abs}[0]{\operatorname{abs}}
\newcommand{pop}[0]{\operatorname{pop}}
\newcommand{aux}[0]{\text{aux}}
\newcommand{opt}[0]{\text{opt}}
\newcommand{tgt}[0]{\text{tgt}}
\newcommand{init}[0]{\text{init}}
\newcommand{lab}[0]{\text{lab}}
\newcommand{rwa}[0]{\text{rwa}}
\newcommand{bra}[1]{\langle#1\vert}
\newcommand{ket}[1]{\vert#1\rangle}
\newcommand{Bra}[1]{\left\langle#1\right\vert}
\newcommand{Ket}[1]{\left\vert#1\right\rangle}
\newcommand{Braket}[2]{\left\langle #1\vphantom{#2}\mid{#2}\vphantom{#1}\right\rangle}
\newcommand{Ketbra}[2]{\vert#1\rangle\!\langle#2\vert}
\newcommand{op}[1]{\hat{#1}}
\newcommand{Op}[1]{\hat{#1}}
\newcommand{dd}[0]{\,\text{d}}
\newcommand{Liouville}[0]{\mathcal{L}}
\newcommand{DynMap}[0]{\mathcal{E}}
\newcommand{identity}[0]{\mathbf{1}}
\newcommand{Norm}[1]{\lVert#1\rVert}
\newcommand{Abs}[1]{\left\vert#1\right\vert}
\newcommand{avg}[1]{\langle#1\rangle}
\newcommand{Avg}[1]{\left\langle#1\right\rangle}
\newcommand{AbsSq}[1]{\left\vert#1\right\vert^2}
\newcommand{Re}[0]{\operatorname{Re}}
\newcommand{Im}[0]{\operatorname{Im}}$

## Model

<center>
    <img src="../figures/3-level_mod.svg" alt="Drawing" width="750">
</center>

The system consists of three levels that evolve, using the rotating wave approximation, according to the relations
$$
\begin{align}
    i \frac{\partial}{\partial t} \Ket{\Psi ^{\,(\pm)} (t)}&= \sum_{i=1}^3 H_i ^{\,(\pm)} (t) \Ket{\Psi ^{\,(\pm)} (t)}\\
\Ket{\Psi ^{\,(\pm)} (t)} &= \sum_{i=1}^3 c_n^{\,(\pm)}(t)\, \Ket{n}.
\end{align}
$$

The different terms of the Hamiltonian can be expressed as

$$
\Op{H}_{1}^{\,(\pm)} = -E_1 (t)\, \,\mu_b^{\pm}\,\begin{pmatrix}
    0 & 1 & 0                          \\
    1 & 0 & 0 \\
    0 & 0 & 0
\end{pmatrix}\,.
$$

$$
\Op{H}_{2}^{\,(\pm)} = -E_2 (t)\, \,\mu_a^{\pm}\,\begin{pmatrix}
    0 & 0 & 0  \\
    0 & 0 & \mathrm{e}^{i\pi/2}  \\
    0 & \mathrm{e}^{-i\pi/2}  & 0
\end{pmatrix}\,.
$$

$$
\Op{H}_{3}^{\,(\pm)} = -E_3 (t)\, \,\mu_c^{\pm}\,\begin{pmatrix}
    0 & 0 & 1                          \\
    0 & 0 & 0 \\
    1 & 0 & 0
\end{pmatrix}\,,
$$

where $E_1(t)$, $E_2(t)$ and $E_3(t)$ are the envelopes of fields with central frequencies $\omega$, $\delta\omega$ and $\omega + \delta\omega$, respectively.
The interesting part of the dynamics comes from the relations between the dipole moments of the two enantiomers, namely $\mu_a^{\,(+)} =  \mu_a^{\,(-)}$, $\mu_b^{\,(+)} =  \mu_b^{\,(-)}$ and $\mu_c^{\,(+)} =  -\mu_c^{\,(-)}$.

This implies that $\Op{H}_{1}^{\,(+)} = \Op{H}_{1}^{\,(-)}$, $\Op{H}_{2}^{\,(+)} = \Op{H}_{2}^{\,(-)}$ and $\Op{H}_{3}^{\,(+)} = -\Op{H}_{3}^{\,(-)}$.


Our goal will be to optimize pulses $E_1(t)$, $E_2(t)$ and $E_3(t)$ such that for the initial states $\Ket{\Psi ^{\,(+)} (0)} =: \Ket{\Psi_{\init}^+} = \Ket{1}$ and $\Ket{\Psi ^{\,(-)} (0)}  =: \Ket{\Psi_{\init}^-}  = \Ket{1}$ and a previously defined final time $T$ we respectively reach the target state $\Ket{\Psi ^{\,(+)} (T)}  =: \Ket{\Psi_{\tgt}^{\,+}} = e^{i \Phi_+}\Ket{2}$ and $\Ket{\Psi ^{\,(-)} (T)}  =: \Ket{\Psi_{\tgt}^{\,-}} = e^{i \Phi_-}\Ket{3}$ for some unspecified phases $ \Phi_+$ and $ \Phi_-$.
That is, although both enantiomers start in the same state, they end up in entirely different states after interaction with the laser fields. Since the phases $ \Phi_+$ and $ \Phi_-$ do not affect this physical optimization goal, it is sufficient to only consider the populations in the following.

First, we define the various dipole moments.

In [ ]:
μa =  2.0  #Note that μa := μa_p = μa_m, and similarly for b
μb =  3.0
μc_p = 0.5
μc_m = -0.5

Next, we define the times at which the pulses start and stop in our initial guesses.

In [ ]:
u_1_start = 0.0 # as a factor of the overall time
u_1_stop  = 1.0 # as a factor of the overall time
u_2_start = 0.0 # as a factor of the overall time
u_2_stop  = 1.0 # as a factor of the overall time
u_3_start = 0.0 # as a factor of the overall time
u_3_stop  = 1.0 # as a factor of the overall time

This allows to define the three pulses interacting with the system.

In [ ]:
def u1(t, args):
    """Guess for the first electric pulse"""
    Amp_1 = .25
    return Amp_1 * krotov.shapes.blackman(
        t, t_start=u_1_start*args["final_t"], t_stop=u_1_stop*args["final_t"]
    )

def u2(t, args):
    """Guess for the second electric pulse"""
    Amp_2 = .25
    return Amp_2 * krotov.shapes.blackman(
        t, t_start=u_2_start*args["final_t"], t_stop=u_2_stop*args["final_t"]
    )

def u3(t, args):
    """Guess for the third electric pulse"""
    Amp_3 = .25
    return Amp_3 * krotov.shapes.blackman(
        t, t_start=u_3_start*args["final_t"], t_stop=u_3_stop*args["final_t"]
    )

Finally, we construct the Hamiltonians.

In [ ]:
def hamiltonian(μa=μa, μb=μb, μc_p=μc_p, μc_m=μc_m):
    """3-level Chiral System Hamiltonian"""

    # Drift H0 and control coupled Hamiltonians for 3-level system
    H0 = qutip.Qobj(
        np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
    )

    H1 = μb * qutip.Qobj(
        np.array([[0.0, 1.0, 0.0], [1.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
    )
    H2 = μa * qutip.Qobj(
        np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1j], [0.0, -1j, 0.0]])
    )

    H3_p = μc_p * qutip.Qobj(
        np.array([[0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [1.0, 0.0, 0.0]])
    )
    H3_m = μc_m * qutip.Qobj(
        np.array([[0.0, 0.0, 1.0], [0.0, 0.0, 0.0], [1.0, 0.0, 0.0]])
    )

    H_p = [H0, [H1, u1], [H2, u2], [H3_p, u3]]
    H_m = [H0, [H1, u1], [H2, u2], [H3_m, u3]]
    return H_p, H_m

H_p, H_m = hamiltonian()

The goal of this example is a state to state transfer with
initial state
$\Ket{\Psi_{\init}^+} = \Ket{\Psi_{\init}^-} = \Ket{1}$ for both enantiomers and target states
$\Ket{\Psi_{\tgt}^{\,+}} = \Ket{2}$ and $\Ket{\Psi_{\tgt}^{\,-}} = \Ket{3}$ (up to a phase) at final time $T$.

In [ ]:
"""Initial and target states"""

psi_init = qutip.Qobj(np.array([1.,0.,0.]))
psi_tgt_p = qutip.Qobj(np.array([0.,1.,0.]))
psi_tgt_m = qutip.Qobj(np.array([0.,0.,1.]))

## Pulses

#### Define the time grid

To view our pulses, we need to define a time-grid, over which we also optimize our pulses later on. Unlike in a parameter optimization, Krotov's algorithm directly optimizes the pulses on the discrete time grid. We will first define the variables `tf`, representing the __final time__ and `nt` describing the __number of time steps__.

In [ ]:
# Final time and number of time steps
tf = 50  # final time T in ns
nt = 200 # number of time steps

# Time grid
tlist = np.linspace(0.,tf, nt)

#### Plot the pulses

After having defined the time grid, let's have a look at the pulses.

In [ ]:
plot_pulse(H_p[1][1], tlist, 'E_1', color="m")
plot_pulse(H_p[2][1], tlist, 'E_2', color="c")
plot_pulse(H_p[3][1], tlist, 'E_3', color="darkgreen")

*Note: `plot_pulse` is defined in `utils/exercise_3_chiral.py`. Use `??plot_pulse` so see its source code*

## Problem 1: Choice of pulse shapes

We have described the pulses using Blackman shapes instead of Gaussian curves, their main difference being that they start and end with an amplitude of 0. Why do you think this is more convenient, especially from an experimental point of view?

In [ ]:
#problem_1.solution

## Simulate dynamics of the guess field

After assuring ourselves that our guess pulses appear as expected, we propagate
the system using our guess.
To track the population in the relevant levels, we first define appropriate projectors $\op{P}_{i} =
\Ketbra{i}{i}$ on the subspaces:

In [ ]:
proj1 = qutip.Qobj(np.diag([1,0,0]))
proj2 = qutip.Qobj(np.diag([0,1,0]))
proj3 = qutip.Qobj(np.diag([0,0,1]))

They will help us later to analyze the population in the different levels.

We also set so called optimization
objectives which contain all the relevant information needed for the propagation:

In [ ]:
objective_p = krotov.Objective(
    initial_state = psi_init, target = psi_tgt_p, H=H_p
)
objective_m = krotov.Objective(
    initial_state = psi_init, target = psi_tgt_m, H=H_m
)

objective_p, objective_m

Having them set up, we can use their **m**aster **e**quation **solve**r function to obtain the dynamics over time. Since no optimization was done so far, we propagate with the guess pulses:

In [ ]:
guess_dynamics_p = objective_p.mesolve(
    tlist, e_ops=[proj1, proj2, proj3],
    args={"final_t":tf}
)

In [ ]:
guess_dynamics_m = objective_m.mesolve(
    tlist,
    e_ops=[proj1, proj2, proj3],
    args={"final_t":tf}
)

Now let's plot the population dynamics:

In [ ]:
plot_population_3lvl(guess_dynamics_p, title="Enantiomer (+)") # function from exercise_3_3WM

What you see here is how the populations of the three levels for enantiomer (+) change during the interaction with the guess pulses.
Remember: Our aim is to get all the population to state 2 and none in state 1!

In [ ]:
plot_population_3lvl(guess_dynamics_m, title="Enantiomer (-)")

And here are the populations for enantiomer (-). As you see, the final state looks very similar to the one of enantiomer (+). There's still work to do to change this!

*Note: `plot_population_3lvl` is defined in `utils/exercise_3_chiral.py`. Use `??plot_population_3lvl` so see its source code*

## Problem 2: Final populations

Try to extract the populations at final time for the (+) enantimer and the (-) enantiomer.  Make sure to use the actual value from the qutip results `guess_dynamics_m` and `guess_dynamics_p`.

In [ ]:
pop1_plus = #
pop2_plus = #
pop3_plus = #

In [ ]:
pop1_minus = #
pop2_minus = #
pop3_minus = #

In [ ]:
problem_2.check([[pop1_plus, pop2_plus, pop3_plus], [pop1_minus, pop2_minus, pop3_minus]])

In [ ]:
#problem_2.hint

In [ ]:
#problem_2.solution

Next, we show how the optimization changes the pulses to reach the desired states.

## Optimization

#### Setting the pulse options

To initialize the optimization we need to provide some guess pulses. When we have no information over the system we can use generic Gaussian-like pulses and see how it affects the population.

On top of that, we need to provide a shape that will determinate which parts of the pulse can be changed. This can ensure, for example, a smooth start and end of the pulse.

In [ ]:
def S(t):
    """Scales the Krotov methods update of the pulse value at the time t"""
    return krotov.shapes.flattop(
        t, t_start=0.0, t_stop=tf, t_rise=0.05*tf, func='sinsq'
    )

We choose an appropriate update factor step size $\frac{1}{\lambda_a}$ for the problem at hand and
make sure Krotov's algorithm considers pulses which start and end with zero amplitude.

In [ ]:
# Changing λ will directly affect how fast the pulse change.

# If it is too big, the iterations will barely change any pulse.
# If it is too small, the pulses may change abruptly.

opt_lambda = 816.0

# Define how the pulses are allowed to change during each iteration
pulse_options = {
    H_p[1][1]: dict(lambda_a=opt_lambda, update_shape=S, args={"final_t":tf}),
    H_p[2][1]: dict(lambda_a=opt_lambda, update_shape=S, args={"final_t":tf}),
    H_p[3][1]: dict(lambda_a=opt_lambda, update_shape=S, args={"final_t":tf})
}

#### Applying Krotov's method

We now supply Krotov's algorithm with all the information it needs to optimize,
consisting of the `objectives` (maximize population for target states at $t_{f}$),
`pulse_options` (the initial shapes of our pulses and how they may be changed)
as well as the `propagator` to use, optimization functional (`chi_constructor`),
`info_hook` (processing occuring inbetween iterations of optimization) and the
maximum number of iterations to perform, `iter_stop`.

We define the variables `J_err_threshold`, `convergence_threshold` and `n_iter`.
The optimization is stopped if either the functional falls below `J_err_threshold` or the update of the functional is smaller than `convergence_threshold`. It will also stop after (at most) `n_iter` optimization steps.

In [ ]:
J_err_threshold = 1e-3
convergence_threshold = 1e-5
n_iter = 10

opt_result = krotov.optimize_pulses(
    [objective_p, objective_m],
    pulse_options,
    tlist,
    propagator=krotov.propagators.expm,
    chi_constructor=krotov.functionals.chis_ss,
    info_hook=krotov.info_hooks.print_table(
        J_T=krotov.functionals.J_T_ss,
    ),
    check_convergence=krotov.convergence.Or(
        krotov.convergence.value_below(J_err_threshold, name='J_T'),
        krotov.convergence.delta_below(convergence_threshold),
        krotov.convergence.check_monotonic_error,
    ),
    iter_stop=n_iter
)

opt_result

Among all the columns of data shown during optimization the most important one is the second column, showing the functional value $J_T$ at the current iteration.
If your optimization stops without reaching the desired value of convergence, have a look at the `ΔJ_T` column which shows just the difference in the functional between two iterations.

We might have found pulse-shapes that fulfill our objective, but what do
they look like?

In [ ]:
plot_pulse(opt_result.optimized_controls[0], tlist, 'E_1', color="m", guess=H_p[1][1])
plot_pulse(opt_result.optimized_controls[1], tlist, 'E_2', color="c", guess=H_p[2][1])
plot_pulse(opt_result.optimized_controls[2], tlist, 'E_3', color="darkgreen", guess=H_p[3][1])

In [ ]:
opt_result.optimized_objectives

Once we have the optimized pulses we can represent the dynamics for the initial states of our objectives.

In [ ]:
opt_dynamics_p = opt_result.optimized_objectives[0].mesolve(
    tlist, e_ops=[proj1, proj2, proj3]
)

plot_population_3lvl(opt_dynamics_p, title="Enantiomer (+)")

In [ ]:
opt_dynamics_m = opt_result.optimized_objectives[1].mesolve(
    tlist, e_ops=[proj1, proj2, proj3]
)

plot_population_3lvl(opt_dynamics_m, title="Enantiomer (-)")

The optimization seems to not quite match our expectations yet. Try to adjust the parameters to improve the convergence.

Remember: For enantiomer (+), only level 2 should populated and for enantiomer (-) only level 3.

## Problem 3: Refining λₐ

Try to adjust the $\lambda_a$ parameter such that you obtain a good result ($J_T < 10^{-3}$) in less than 5 optimization steps.

In [ ]:
#problem_3.hint

In [ ]:
#problem_3.solution

## Next steps

If you are also interested in gate optimization for quantum information applications, 
we invite you to have a look at [Exercise III.4](py_exercise_3_4_gate.ipynb).

Alternatively, you may continue with [further examples from the documentation of the
`krotov` package](https://qucontrol.github.io/krotov/v1.2.1/09_examples.html)
which are beyond the scope of this tutorial. These include many interesting setups, e.g., the
optimization of quantum gates in open quantum systems.

<!-- Autofooter begin -->

---

[⬆︎ jump to top](#navtitle_3_3_py)
<!-- Autofooter end -->